# E07 Ising 临界信号：热容/易感性峰值与 Binder cumulant

> 目标：在有限尺寸模拟中，用“峰值/交点”识别临界附近行为（相变式信号）。

我们计算三个量（每自旋）：
- 热容 proxy：\(C_V \propto \beta^2 \mathrm{Var}(e)\)
- 易感性 proxy：\(\chi \propto \beta \mathrm{Var}(m)\)
- Binder cumulant：\(U = 1 - \langle m^4\rangle / (3\langle m^2\rangle^2)\)


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

from exercises.src.ising import simulate_ising, heat_capacity, susceptibility, binder_cumulant


## 1) 对两个系统尺寸做 β 扫描

In [ ]:
betas = np.linspace(0.25, 0.65, 9)
Ls = [12, 18]

rows = []
for L in Ls:
    for beta in betas:
        tr = simulate_ising(L, beta, n_sweeps=2500, burn_in=700, thin=5, seed=1)
        Cv = heat_capacity(beta, tr.energy)
        chi = susceptibility(beta, tr.magnetization)
        U = binder_cumulant(tr.magnetization)
        rows.append({"L": L, "beta": float(beta), "Cv": Cv, "chi": chi, "binder": U})

df = pd.DataFrame(rows)
df.head()


In [ ]:
# Plot Cv
plt.figure()
for L in Ls:
    sub = df[df["L"] == L]
    plt.plot(sub["beta"], sub["Cv"], marker="o", label=f"L={L}")
plt.xlabel("beta")
plt.ylabel("Cv (proxy, per spin)")
plt.title("Heat capacity proxy vs beta")
plt.grid(True)
plt.legend()
plt.show()


In [ ]:
# Plot susceptibility
plt.figure()
for L in Ls:
    sub = df[df["L"] == L]
    plt.plot(sub["beta"], sub["chi"], marker="o", label=f"L={L}")
plt.xlabel("beta")
plt.ylabel("chi (proxy, per spin)")
plt.title("Susceptibility proxy vs beta")
plt.grid(True)
plt.legend()
plt.show()


In [ ]:
# Plot Binder cumulant
plt.figure()
for L in Ls:
    sub = df[df["L"] == L]
    plt.plot(sub["beta"], sub["binder"], marker="o", label=f"L={L}")
plt.xlabel("beta")
plt.ylabel("Binder cumulant U")
plt.title("Binder cumulant vs beta")
plt.grid(True)
plt.legend()
plt.show()


In [ ]:
# Very rough "peak" estimates (finite-size)
peak_Cv = df.loc[df.groupby("L")["Cv"].idxmax(), ["L","beta","Cv"]].sort_values("L")
peak_chi = df.loc[df.groupby("L")["chi"].idxmax(), ["L","beta","chi"]].sort_values("L")

peak_Cv, peak_chi


## 讨论（写在你的记录里）
- 峰值位置随 L 是否有漂移？这就是有限尺寸效应的直观表现。
- Binder 曲线是否在某个 β 区间接近交点？你会如何用它估计临界附近区间？
- 为什么这里称它们是“proxy”？（提示：有限尺寸、采样误差、自相关）